In [2]:
from bs4 import BeautifulSoup
import requests
from requests import get
import json
import random
import string
import sqlite3
import logging
import re


In [44]:
flickfilosopher_file = '../Scraping/flickfilosopher.txt'
current_reviews = set(line.strip() for line in open(flickfilosopher_file))

db_path = '../Data/test.db'
db_name = 'test'
conn = sqlite3.connect(db_path)
c = conn.cursor()

c.execute(("""
       CREATE TABLE IF NOT EXISTS
           test(
               NODE TEXT,
               URL TEXT,
               REVIEWBODY TEXT,
               RATING TEXT,
               REVIEWRATING TEXT,
               BESTRATING TEXT,
               WORSTRATING TEXT,
               PRIMARY KEY (NODE, URL))
   """))

In [45]:
def generateNode(length):
    letters_and_digits = string.ascii_letters + string.digits
    result_str = ''.join((random.choice(letters_and_digits) for i in range(length)))
    node = "_:znode" + result_str
    return node

# all moviereviews
url = "https://www.flickfilosopher.com/all-reviews-a-m"
# page2 = "https://www.flickfilosopher.com/all-reviews-n-z"
# url_lst = ['https://www.flickfilosopher.com/all-reviews-a-m', 'https://www.flickfilosopher.com/all-reviews-n-z']
# for url in url_lst:
#     ...

soup = BeautifulSoup(get(url).text, 'html.parser')
movie_container = soup.find_all("div", class_="entry post clearfix")
for link in movie_container[0].find_all("a"):
    print(link.get('href'))

# print(movie_container)
taxo_pattern = re.compile("</a><br/>", re.IGNORECASE)



None
#A
#B
#C
#D
#E
#F
#G
#H
#I
#J
#K
#L
#M
https://www.flickfilosopher.com/all-reviews-n-z
None
https://www.flickfilosopher.com/2002/10/abandon-review.html
https://www.flickfilosopher.com/2001/11/abcd-review.html
https://www.flickfilosopher.com/2011/09/abduction-review.html
https://www.flickfilosopher.com/2013/02/oscar-nominated-animated-shorts-85th-academy-awards-review.html
https://www.flickfilosopher.com/2019/10/abominable-movie-review-a-bit-of-a-snow-job.html
https://www.flickfilosopher.com/2002/05/about-a-boy-review.html
https://www.flickfilosopher.com/2013/09/time-review-creep-trick.html
https://www.flickfilosopher.com/2012/06/abraham-lincoln-vampire-hunter-review.html
https://www.flickfilosopher.com/2016/04/the-keeper-of-lost-causes-kvinden-i-buret-and-the-absent-one-fasandraeberne-movies-review-good-clean-nasty-nordic-noir.html
https://www.flickfilosopher.com/2015/08/absolutely-anything-movie-review-absolutely-appalling.html
https://www.flickfilosopher.com/2004/08/absolutely-f